In [13]:
import ee
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob
from datetime import date

from uuid import uuid4


from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection
import plotly
import plotly.express as px
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import geopandas as gpd
import seaborn as sns

import dask.dataframe as dd
import geopandas as gpd
from heatwave_1 import detect
from datetime import timedelta
import seaborn as sns
from datetime import date
import matplotlib.pyplot as plt

sns.set_style('darkgrid')

In [27]:
daily_max_dic = {}

for state in ['California', 'Oregon', 'Washington', 'British Columbia']:
            
    daily_max_files = glob.glob(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/{state}/max*')
    
    temp = dd.read_csv(daily_max_files)
    temp = temp.compute()
    daily_max_dic[state] = temp
    

full_df = pd.concat(daily_max_dic.values())

full_df['datetime'] = pd.to_datetime(full_df[['day', 'month', 'year']])
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]

full_df['location'] = full_df['county'] + ', ' + full_df['state']
full_df.drop_duplicates(['location', 'datetime'], inplace=True)

full_df = full_df[(full_df['month'] != 10)]
full_df = full_df[(full_df['month'] != 4)]

full_df['pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

# Identify heatwaves

In [35]:
#create dictionary containing each county's heatwave periods
heatwave_dic = {}
for county in full_df['location'].unique():
    county_df = full_df[(full_df['location'] == county)]
    county_df.reset_index(inplace=True)
    county_df.sort_values(by='datetime', inplace=True)
    county_df.drop_duplicates(['county', 'datetime'], inplace=True)
    
    #call 'detect' function from heatwave_1.py
    heatwave_dic[county] = detect(county_df['datetime_ord'].values, county_df['temperature_2m'].values, pctile=90, climatologyPeriod = [None, None], minDuration=2)


#create dictionary of dates for each heatwave
heatwave_dates = {}

#create dictionary of heatwave start day and lengths
consec_dates = {}

#generate unique ID code for each heatwave

heatwave_IDs = {}
for county in heatwave_dic.keys(): 
    consec_dates[county] = {}
    heatwave_dates[county] = []
    heatwave_IDs[county] = {}


    heatwave_starts = heatwave_dic[county][0]['date_start']
    heatwave_ends = heatwave_dic[county][0]['date_end']
    for i in range(len(heatwave_starts)):
        heatwave_code = str(uuid4())
        heatwave_days = pd.date_range(heatwave_starts[i],heatwave_ends[i],freq='d')
        length_heatwaves = len(heatwave_days)
        consec_dates[county][heatwave_days[0]] = length_heatwaves
        for item in heatwave_days:
            heatwave_dates[county].append(item)
            heatwave_IDs[county][item] = heatwave_code
            
            
heat_df = pd.DataFrame.from_dict(heatwave_dates, orient='index').unstack().reset_index()
heat_df.rename(columns={'level_1':'location', 0:'datetime'}, inplace=True)


heat_df.dropna(inplace=True)
heat_df['heatwave']=1

/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_81532/2444538110.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_df.sort_values(by='datetime', inplace=True)
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_81532/2444538110.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county_df.drop_duplicates(['county', 'datetime'], inplace=True)
/var/folders/8g/qp1k_1g91m5f2m9pypxwxdhh0000gn/T/ipykernel_81532/2444538110.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [36]:
#merge full temperature dataset with identified 
full_df = full_df.merge(heat_df, on=['location', 'datetime'], how='left')

full_df['heatwave'] = full_df['heatwave'].replace(np.nan, 0)

In [37]:
full_df['pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

In [38]:
full_df[full_df['heatwave'] == 1].groupby('state')['temperature_2m'].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
British Columbia,2759.0,31.107162,3.088932,23.677606,28.885011,30.821664,33.019601,44.067896
California,5309.0,40.125608,4.056782,26.468729,37.697031,40.246210,42.695184,51.490472
Oregon,3218.0,36.244454,2.850756,28.346628,34.471010,36.019327,37.955442,48.115305
Washington,3569.0,35.105173,4.554628,20.246011,31.810403,35.384897,38.426370,48.115305


In [39]:
#create dictionary of unique heatwave ID codes
codes = pd.DataFrame.from_dict(heatwave_IDs, orient='index').unstack().reset_index()

In [40]:
codes.columns=['datetime', 'location', 'heat_ID']

In [41]:
codes.dropna(inplace=True)

In [42]:
codes.head()

,datetime,location,heat_ID
0,2016-06-19,"San Bernardino, California",4f54d330-c4d6-4284-a98d-faac3932053a
1,2016-06-19,"Imperial, California",280fba6d-fc57-437e-929c-892a15831d66
2,2016-06-19,"Los Angeles, California",849c639b-2e10-441b-a3a4-5b9d1af8cd8b
3,2016-06-19,"Orange, California",022f5a56-2f65-4e95-badc-3ebf4b5ed443
4,2016-06-19,"Riverside, California",39319368-fd0d-408d-9f13-13feda33f09a


In [43]:
#get dataframe of heatwave durations
consec = pd.DataFrame.from_dict(consec_dates, orient='index').unstack().reset_index()

In [44]:
consec.columns= ['datetime', 'location', 'Heat Wave Duration']

In [45]:
consec['Heat Wave Duration'].max()

12.0

In [46]:
consec.head()

,datetime,location,Heat Wave Duration
0,2016-06-19,"San Bernardino, California",3.0
1,2016-06-19,"Imperial, California",2.0
2,2016-06-19,"Los Angeles, California",3.0
3,2016-06-19,"Orange, California",2.0
4,2016-06-19,"Riverside, California",2.0


In [47]:
consec.dropna(inplace=True)

In [48]:
full_df = full_df.merge(codes, on=['datetime', 'location'])

In [49]:
full_df = full_df[full_df['year'] >= 2020]



In [50]:
full_df = full_df.merge(consec, on=['datetime', 'location'], how='left')

In [51]:
heatwaves = full_df.copy()

In [52]:
heatwaves['location'] = heatwaves['county'] + ', ' + heatwaves['state']

In [53]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [54]:
heatwaves = heatwaves.merge(consec, on=['datetime', 'location'], how='left')

In [55]:
heatwaves = heatwaves[heatwaves['month'] != 10]

In [56]:
heatwaves.sort_values(by='datetime', inplace=True)

In [59]:
heatwaves.groupby('state')['temperature_2m'].describe()

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
British Columbia,1351.0,31.722341,3.416595,23.763834,29.124216,31.531015,33.779840,44.067896
California,2830.0,40.442041,3.986818,26.468729,37.979551,40.615366,42.978700,51.490472
Oregon,1671.0,36.666122,3.055238,28.411096,34.814584,36.386942,38.484155,48.115305
Washington,1717.0,36.057898,4.830247,20.246011,32.816537,36.417383,39.459253,48.115305


In [57]:
#save file
heatwaves.to_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwave_clean_90_v1.csv')